# Reto Optimización no lineal

In [1]:
import pandas as pd
import numpy as np


In [2]:
distancias_poligonos = pd.read_csv('distanciasMatricesCoordenadas.csv', encoding='latin1')

In [3]:
# Eliminar la primer columna que es un índice
distancias_poligonos = distancias_poligonos.drop(columns=['Polí­gono'])
distancias_poligonos.head(10)	

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,0.000000,12.093387,9.219544,6.500000,6.500000,17.442764,21.569655,18.309833,16.552945,15.206906,...,31.953091,31.674122,31.064449,37.396524,43.410828,41.382363,40.205721,39.080046,36.912100,22.940139
1,12.093387,0.000000,3.500000,8.000000,13.000000,5.385165,10.295600,9.000000,9.656604,12.041595,...,24.005208,25.317978,26.405492,39.284221,45.675486,32.268406,31.752953,31.503968,30.907119,16.492422
2,9.219544,3.500000,0.000000,4.500000,9.500000,8.732100,12.379418,9.656604,9.000000,10.062306,...,24.331050,25.005000,25.495098,36.776351,43.133500,33.203915,32.380550,31.752953,30.536863,16.007811
3,6.500000,8.000000,4.500000,0.000000,5.000000,13.152946,15.811388,12.041595,10.062306,9.000000,...,25.460754,25.317978,25.005000,33.811980,40.078049,34.889110,33.708308,32.626676,30.647186,16.492422
4,6.500000,13.000000,9.500000,5.000000,0.000000,18.110770,20.124612,15.811388,13.086300,10.295600,...,27.536340,26.570661,25.401772,30.955613,37.030393,37.312866,35.794553,34.270979,31.531730,18.357560
5,17.442764,5.385165,8.732100,13.152946,18.110770,0.000000,7.000000,8.602325,11.011358,14.764800,...,22.455512,24.698178,26.669271,41.907637,48.303727,29.567719,29.516944,29.841247,30.203477,16.643317
6,21.569655,10.295600,12.379418,15.811388,20.124612,7.000000,0.000000,5.000000,8.500000,13.000000,...,15.660460,18.357560,20.934421,38.239378,44.533695,22.588714,22.522211,22.945588,23.711811,11.401754
7,18.309833,9.000000,9.656604,12.041595,15.811388,8.602325,5.000000,0.000000,3.500000,8.000000,...,15.008331,16.492422,18.117671,33.871100,40.227478,23.563743,22.852790,22.505600,22.073740,8.062300
8,16.552945,9.656604,9.000000,10.062306,13.086300,11.011358,8.500000,3.500000,0.000000,4.500000,...,15.524175,16.007811,16.763055,30.927334,37.316216,24.829418,23.717082,22.852790,21.552262,7.017834
9,15.206906,12.041595,10.062306,9.000000,10.295600,14.764800,13.000000,8.000000,4.500000,0.000000,...,17.240940,16.492422,16.007811,27.335874,33.737961,27.041635,25.500000,24.052027,21.708293,8.062300


In [4]:
# Convertir el DataFrame a una matriz NumPy
matriz_distancias = distancias_poligonos.to_numpy()

In [5]:
print(matriz_distancias[0, 2])  
#(fila, columna) = (0,2) = Distancia entre poligono 1 y poligono 3
# Verificar el tipo de dato de la matriz
# Por lo tanto, distancia entre poligono i al poligono j es:
# matriz_distancias[i-1, j-1] = distancia entre poligono i y poligono j
print(matriz_distancias[17,5]) #distancia entre poligono 18 y poligono 6

9.219544457
28.42534081


### Definición de parámetros:

In [6]:
tiempo_carga = 30 #30 minutos
tiempo_descarga = 30
jornada_laboral = 6 #6 horas
almacenamiento = 400 #Almacenamiento inventario en mt2
costo_transporte = 4500 #Costo de la entrega de los proveedores
capacidad_traslado_max = 8000 #Traslado máximo de unidades
costo_plantacion = 20 #Costo de la plantación por unidad
dias_anticipación = 1 # Días de anticipación para realizar el pedido
tratamiento_nopales = 0.333 # Como se trabaja en horas , esto equivale a 20 minutos
min_dias_aclimatacion = 3 # Mínimo de días de aclimatación
max_dias_aclimatacion = 7 # Máximo de días de aclimatación
M = 1000000000 # Un número grande para representar un costo alto/imposible en la matriz de costos
# Especies de plantas:
labels = {
1: 'Agave Lechuguilla', 
2:'Agave Salmiana', 
3:'Agave Scabra', 
4:'Agave Striata', 
5:'Opuntia Cantabrigiensis', 
6:'Opuntia Engelmanni', 
7:'Opuntia Robusta', 
8:'Opuntia Streptacanta', 
9:'Prosopis Laevigata', 
10:'Yucca Filifera'
}
# Matriz de costos de especies por proveedor:
costos_especie_proveedor = [M,M,M,26,
                      M,M,M,26,
                      M,26,M,26,
                      M,26,25,M,
                      M,17,18,M,
                      M,M,18,21,
                      M,17,18,18,
                      M,M,18,M,
                      26.5,M,M,M,
                      26,M,M,M]
# Distancias entre polígonos: matriz_distancias
# Demanda de especies por polígono:
hectareas_poligono = {
    1: 5.4,
    2:0,
    3: 8,
    4: 8,
    5: 7.56,
    6: 0,
    7: 0,
    8: 0,
    9: 0,
    10: 0,
    11: 0,
    12: 0,
    13: 0,
    14: 0,
    15: 0,
    16: 5.64,
    17: 6.11,
    18: 7.11,
    19: 4.92,
    20: 1.38,
    21: 8,
    22: 7.82,
    23: 5.53,
    24: 5.64,
    25: 5.05,
    26: 4.75,
    27: 1.28,
    28: 6.64,
    29: 6.54,
    30: 6.76,
    31: 0
}
demanda_especies_por_hectarea = {
    1: 33,
    2: 157,
    3: 33,
    4: 33,
    5: 39,
    6: 30,
    7: 58,
    8: 51,
    9: 69,
    10: 21
}

### Procesos: Algoritmo genético

In [7]:
#Input: Dias a calcular la planeación (opcional, sino establecer un valor por defecto)
dias_planeacion = 30  # Por defecto, 30 días de planeación
#Output: 
# 1.- Lista con las cantidades a pedir a cada proveedor por día
# 2.- Lista con los viajes realizados por día, mostrando la ruta (poligonos visitados por viaje)
# 3.- Una actualización de cuántas especies hay en cada polígono al final de los días de planeación: Se puede resumir en cuántas especies se plantaron al final del día
# Igual puede mostrarse cuánta demanda se atendió al final del día
# 4.- Mostrar información de las distancias recorridas en cada viaje, así como el costo total de la planeación

In [8]:
def calcular_demanda_inicial_especies():
    pass

# A que proveedor le conviene comprar (y cuando) cada especie para satisfacer la demanda de los polígonos en total en el tiempo t (en determinado dia).
def calcular_pedidos_proveedor(demanda_especies):
    pedidos_proveedor = {}
    for especie, demanda in demanda_especies.items():
        # Encontrar el proveedor con el costo más bajo para la especie
        min_costo = M
        proveedor_elegido = None
        for proveedor, costo in enumerate(costos_especie_proveedor):
            if costo < min_costo:
                min_costo = costo
                proveedor_elegido = proveedor + 1  # Ajustar índice a 1-based
        pedidos_proveedor[especie] = (proveedor_elegido, demanda)
    return pedidos_proveedor

# Será mejor manejar el tiempo en horas para evitar confusiones con los días y minutos.
# Que especies de plantas se deben plantar en cada polígono para satisfacer la demanda de los polígonos, considerando que 
# las plantas requieren un tiempo de aclimatación (entre 3-7 días)
# Los nopales (Opuntia) requieren un tratamiento especial de 20 minutos por planta antes de ser enviados a los polígonos.
# Se tiene un inventario disponible determinado por la cantidad de plantas que te llegan y las que llevas a plantar
# Se tiene un tiempo de carga y descarga de 30 minutos cada uno, y una jornada laboral de 6 horas.
# Se deben de optimizar los costos de transporte y plantación
# Buscar la mejor forma de satisfacer la demanda de los polígonos en el tiempo t (en determinado dia) considerando las restricciones de tiempo, inventario y costos.
# Buscar optimizar los tiempos tambien


In [9]:
# Definir los índices de filas y columnas a conservar
indices_conservar = [0, 2, 3, 4] + list(range(15, 30))  # [0, 2, 3, 4, 15, 16, ..., 29]

# Verificar que tenemos el número correcto de índices (debería ser 19)
print(f"Número de índices a conservar: {len(indices_conservar)}")

# Crear una nueva matriz reducida tomando solo las filas y columnas seleccionadas
distancias_reducida = distancias_poligonos.iloc[indices_conservar, indices_conservar]

# Verificar que la matriz resultante es 19x19
print(f"Forma de la matriz reducida: {distancias_reducida.shape}")

# Mostrar la matriz reducida
distancias_reducida.head()
distancias_reducida.to_csv('distancias_reducidas_19_poligonos.csv', index=False, encoding='latin1')

Número de índices a conservar: 19
Forma de la matriz reducida: (19, 19)
